## Jupyterhub on Kubernetes

This notebook can be used to launch a standard instance of Jupyterhub which includes a proxy server, and an instance of Jupyterhub. 

Notes on the Server: 
`/etc/jupyterhub/config` is where the configuration is for jupyterhub. 



In [3]:
cf = dict()
cf['cloud_provider']='azure'      #azure of google
cf['project_name']='kuberlytics'  #Must be globally unique.
cf['restore_config']=False   #Only True for existing or previous cluster   
cf['cluster_name']='kuberlytics'         #Only use if restore_config==True
cf['config_file']='config/config.yaml'  #The configuration will be overwritten if retreived.
cf['path']='/home/jovyan/launch/'  #Path to admintools repository
cf['ssh_dir']='/home/jovyan/.ssh/'      #ssh directory

In [4]:
import sys 
import importlib
import ruamel.yaml
sys.path.append(cf['path']+"lib/kuberutils") #Adds a local library path.
import kuberutils as ku   #This imports some utilities.
importlib.reload(ku)      #This just reloads the utilities.
cf=ku.initialize(cf)  #This initializes a configuration object.

### Print Configuration
Optionally you can print the configuration and common commands for your desired cluster. You can use this as a reference and copy and paste into the terminal.

In [5]:
#This will print common commands for your clustl
print(ruamel.yaml.dump(cf, sys.stdout, Dumper=ruamel.yaml.RoundTripDumper))

#Where are you running notebooks? This is used for path.
project_name: kuberlytics   #must be globably unique project
cloud_provider: azure   #google or azure
cluster_name: kuberlytics
restore_config: false
path: /home/jovyan/launch/
config_file: config/config.yaml
ssh_dir: /home/jovyan/.ssh/

#Start of Google Specific Configuration
g_project: kuberlytics             #Google project name
g_cluster_name: kuberlytics        #Name of your Google Cluster
g_region: us-east1                 #Selection from gcloud compute regions list.
g_zone: us-east1-b                  #Selection from gcloud compute regions list.
g_machine_type: n1-highmem-4        #Type of Server
g_account: you@mail.com    #Email Associated with the account.
g_authorization_file: auth.json     #Service account authorization file.
g_service_account_name: kuberlytics  #Service account name.
g_fixedip_namespace: jupyterhub-kuberlytics
g_num_nodes: 1                      #The default number of nodes (servers)
g_num_nodes_class

### Set Jupyterhub Config
This will setup a basic configuration file for installing Jupyterhub. 

In [ ]:
cf=ku.set_jupyterhub_config(cf)

## Download Helm Repo

In [ ]:
#Add the jupyterhub repo.
print(ku.bash_command('jup_repo',cf))

In [ ]:
#This will install the jupyterhub instance.
print(ku.bash_command('jup_install',cf))

In [10]:
#This will check on the status of Jupyterhub.
print(ku.bash_command('jup_describe', cf))

Executing jup_describe:
 kubectl --namespace=kuberlytics get pod
NAME                             READY     STATUS    RESTARTS   AGE
hub-deployment-454936357-3r3kr   1/1       Running   0          34s
proxy-3278366498-h9mn7           2/2       Running   0          20m



In [ ]:
#Get the ip/url. 
#cf=ku.get_jupyterhub_ip(cf)
ku.get_jupyterhub_ip(cf)

## Update Authorization
When just starting out, the default configuration launches with no password.  While there is still a username and login screen, 


In [6]:
#This is how we set the authororization
#ku.update_config('auth','jup_github_auth', cf)
#ku.update_config('auth','jup_google_auth', cf)
ku.update_config('auth','jup_dummy_auth', cf)


In [7]:
#Add Administrators
ku.update_config('admin','jup_admin', cf)

### Update the Singleuser Container
This can allow you to utilize a different container. 


In [8]:
ku.update_config('singleuser','jup_singleuser', cf)

In [9]:
#Upgrading Jupyterhub 
print(ku.bash_command('jup_upgrade',cf))

Executing jup_upgrade:
 helm upgrade kuberlytics jupyterhub/jupyterhub --version=v0.5.0-e154ab7 -f /home/jovyan/launch/config/jupyterhub/kuberlytics/config.yaml
Release "kuberlytics" has been upgraded. Happy Helming!
LAST DEPLOYED: Fri Nov 17 13:52:57 2017
NAMESPACE: kuberlytics
STATUS: DEPLOYED

RESOURCES:
==> v1/ConfigMap
NAME        DATA  AGE
hub-config  21    20m

==> v1/PersistentVolumeClaim
NAME        STATUS  VOLUME                                    CAPACITY  ACCESSMODES  STORAGECLASS  AGE
hub-db-dir  Bound   pvc-e54500ba-cb9b-11e7-b7e7-000d3a002e1a  1Gi       RWO          default       20m

==> v1/Service
NAME          CLUSTER-IP    EXTERNAL-IP    PORT(S)                     AGE
hub           10.0.151.72   <none>         8081/TCP                    20m
proxy-public  10.0.252.113  13.68.116.181  80:32126/TCP,443:30145/TCP  20m
proxy-api     10.0.194.195  <none>         8001/TCP                    20m
proxy-http    10.0.176.193  <none>         8000/TCP                    20m

==

### Backup Jupyterhub file. 

In [ ]:
ku.backup_file_az(cf['cluster_name']+"jupyter-config-file", cf['path']+"/config/cf['config_file'], cf)

Adding SSL Support via Let's Encrypt
helm repo update 
helm install --name=letsencrypt --namespace=kube-system stable/kube-lego \
             --set config.LEGO_EMAIL=<your-email> \
             --set config.LEGO_URL=https://acme-v01.api.letsencrypt.org/directory

## Cleanup the Installation
This will cleanup the installation, deleting the instance of Jupyterhub.

In [ ]:
#Delete Jupyterhub with Helm.
print(ku.bash_command('jup_delete_instance',cf))

In [ ]:
#Delete Jupyterhub with Helm.
print(ku.bash_command('jup_delete_namespace',cf))

### Backup Config,
If you make any changes to the main config file, this backs it up. 

In [ ]:
print(ku.backup_kube(cf))